In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa

dir = './data'
file = '2021_Final_Assisted_Reproductive_Technology__ART__Summary_20240307.csv'

df = pd.read_csv(f'{dir}/{file}')
df.columns

Index(['Year', 'LocationAbbr', 'LocationDesc', 'FacilityName',
       'MedicalDirector', 'Address', 'City', 'ZipCode', 'Phone',
       'Clinic Status', 'Topic', 'SubTopic', 'Question', 'Breakout_Category',
       'Breakout', 'Data_Value', 'data_value_num',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'Cycle_Count',
       'ClinicId', 'DisplayOrder', 'TopicId', 'SubTopicId', 'QuestionId',
       'BreakoutCategoryId', 'BreakoutId', 'Geolocation'],
      dtype='object')

In [2]:

cols_to_use = ['Year', 'LocationAbbr', 'LocationDesc', 'FacilityName',
               'ClinicId', 'Question', 'QuestionId',
               'data_value_num', 'Breakout_Category', 'Breakout'
               ]
df = df[cols_to_use]

print(df['Question'].drop_duplicates().to_string())


0                                      Data Verified By:
1                          Number of intended retrievals
5      Average number of intended retrievals per live...
9      Percentage of intended retrievals resulting in...
13     Percentage of intended retrievals resulting in...
17                                  Number of retrievals
21     Percentage of retrievals resulting in live-bir...
25     Percentage of retrievals resulting in singleto...
29                                   Number of transfers
33     Percentage of transfers resulting in live-birt...
37     Percentage of transfers resulting in singleton...
41     Percentage of new patients having live-birth d...
45     Percentage of new patients having live-birth d...
49     Percentage of new patients having live-birth d...
53     Average number of intended retrievals per new ...
57     Average number of transfers per intended retri...
73                                Total number of cycles
78     Percentage of intended e

In [3]:
df.loc[:, ['ClinicId', 'QuestionId']].drop_duplicates().groupby('ClinicId').count()

,QuestionId
ClinicId,
1,48
2,48
3,48
4,48
6,48
...,...
889,48
892,48
899,48


In [4]:
by_state_counts = (df.loc[:, ['ClinicId', 'LocationAbbr']].drop_duplicates()
                     .groupby('LocationAbbr').count().sort_values(by='ClinicId')
                   )
print(by_state_counts.iloc[:10, :])
print(by_state_counts.iloc[-2:,:])

              ClinicId
LocationAbbr          
AK                   1
AR                   1
SD                   1
RI                   1
NH                   1
ND                   1
MT                   1
ME                   1
ID                   1
VT                   2
              ClinicId
LocationAbbr          
NY                  45
CA                  78


In [5]:
qs = df[['QuestionId', 'Question']].drop_duplicates()
with pd.option_context('display.max_colwidth', None):
    print(qs.to_string(index=False))
    print(qs.query('Question.str.contains("after 1")').to_string())

QuestionId                                                                                 Question
      Q400                                                                        Data Verified By:
      Q301                                                            Number of intended retrievals
      Q302                            Average number of intended retrievals per live-birth delivery
      Q303                     Percentage of intended retrievals resulting in live-birth deliveries
      Q304           Percentage of intended retrievals resulting in singleton live-birth deliveries
      Q305                                                                     Number of retrievals
      Q306                              Percentage of retrievals resulting in live-birth deliveries
      Q307                    Percentage of retrievals resulting in singleton live-birth deliveries
      Q308                                                                      Number of transfers


In [6]:
df.query('QuestionId == "Q311"')

,Year,LocationAbbr,LocationDesc,FacilityName,ClinicId,Question,QuestionId,data_value_num,Breakout_Category,Breakout
41,2021,FL,Florida,"Fertility & IVF Center of Miami, Inc., Fertili...",241,Percentage of new patients having live-birth d...,Q311,49.0,Age of Patient,<35
42,2021,FL,Florida,"Fertility & IVF Center of Miami, Inc., Fertili...",241,Percentage of new patients having live-birth d...,Q311,40.0,Age of Patient,35-37
43,2021,FL,Florida,"Fertility & IVF Center of Miami, Inc., Fertili...",241,Percentage of new patients having live-birth d...,Q311,24.3,Age of Patient,38-40
44,2021,FL,Florida,"Fertility & IVF Center of Miami, Inc., Fertili...",241,Percentage of new patients having live-birth d...,Q311,11.5,Age of Patient,>40
175,2021,IL,Illinois,Vios Fertility Institute-Crest Hill,261,Percentage of new patients having live-birth d...,Q311,48.0,Age of Patient,<35
...,...,...,...,...,...,...,...,...,...,...
60785,2021,NaN,National,National,9999,Percentage of new patients having live-birth d...,Q311,40.1,Age of Patient,35-37
60786,2021,NaN,Other,National,9999,Percentage of new patients having live-birth d...,Q311,25.8,Age of Patient,38-40
60787,2021,NaN,National,National,9999,Percentage of new patients having live-birth d...,Q311,25.8,Age of Patient,38-40
60788,2021,NaN,Other,National,9999,Percentage of new patients having live-birth d...,Q311,8.5,Age of Patient,>40


In [7]:
by_clinic = (
    df[['FacilityName', 'LocationAbbr', 'QuestionId', 'data_value_num']]
)
retrievals_transfers = (
    by_clinic.query('QuestionId == "Q305" or QuestionId == "Q308"')
              .groupby(['FacilityName', 'LocationAbbr', 'QuestionId'])
              .sum().unstack(level='QuestionId')
)
              
retrievals_transfers.columns = ['Retrievals', 'Transfers']
retrievals_transfers



,,Retrievals,Transfers
FacilityName,LocationAbbr,,
"ACFS Fertility, Arizona Associates for Reproductive Health",AZ,151.0,86.0
AHN Center for Reproductive Medicine,PA,175.0,134.0
"Advanced Fertility Associates Medical Group, Inc.",CA,78.0,95.0
"Advanced Fertility Care, PLLC",AZ,220.0,165.0
Advanced Fertility Center of Chicago,IL,525.0,532.0
...,...,...,...
Wisconsin Fertility Institute,WI,148.0,198.0
Womack Army Medical Center,NC,75.0,101.0
Women & Infants Fertility Center,RI,364.0,371.0


In [8]:
(retrievals_transfers.query('LocationAbbr == "AL"')[['Retrievals', 'Transfers']]
     .sum().rename('Totals')
 )

Retrievals    486.0
Transfers     467.0
Name: Totals, dtype: float64

In [9]:
live_birth_questions = ['Q311', 'Q312', 'Q313']
live_birth_category_means = (
    by_clinic.query('QuestionId in @live_birth_questions').drop(columns=['FacilityName'])
             .groupby(['LocationAbbr', 'QuestionId']).mean().unstack(level='QuestionId')
)
live_birth_category_means.columns = ['1 xfer', '1 or 2 xfers', 'all xfers']
with pd.option_context('display.max_rows', None):
    display(live_birth_category_means)

,1 xfer,1 or 2 xfers,all xfers
LocationAbbr,,,
AK,0.000000,0.000000,0.000000
AL,9.500000,10.565000,10.565000
AR,21.525000,23.325000,23.325000
AZ,21.051923,23.559615,24.032692
CA,14.701282,17.671154,18.632692
CO,24.332143,28.100000,28.796429
CT,27.220833,32.129167,33.895833
DC,20.475000,24.237500,25.337500
DE,21.100000,23.037500,23.037500
